# Reinforcement learning with Foolsball
- Reinforcement learning is learning to make decisions from experience.
- Games are a good testbed for RL.
 

# About Foolsball
- 5x4 playground that provides a football/foosball-like environment.
- A controllable player:
  - always spawned in the top-left corner
  - displayed as '⚽'
  - can move North, South, East or West.
  - can be controlled algorithmically
- A number of **static** opponents, each represented by 👕, that occupy certain locations on the field.
- A goalpost 🥅 that is fixed in the bottom right corner

## Goals
### Primary goal
- We want the agent to learn to reach the goalpost 

### Secondary goals
- We may want the agent to learn to be efficient in some sense, for example, take the shortest path to the goalpost. 

## Some definitions to make our enviroment look pretty

In [ ]:
agent = '⚽'
opponent = '👕'
goal = '🥅'

arena = [['⚽', ' ' , '👕', ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , '🥅']]

# Implementing an environment for the game of Foolsball
- Text environments are simple to render in a notebook and super-fast to experiment with.
- We want to build out own environment for two reasons:
  - It's a great exercise in understanding the finer details, like states, actions, rewards, returns.
  - Some of the experimentation we do requires looking under the hood of the environment, which is easier with your own implementation than a third-party environment OpenAI Gym.
  - Environment like OpenAI Gym has a simple `step(), reset()` API that we also implement. So porting our implementation over to Gym should be easy (and fun)!



# Let's start!
---

# Step 1: Build the Foolsball environment
The code below provides a skeleton for the **Foolsball** environment we want our agent to train in. Fill in the code marked with #Todo to create a working environment.

1. Go to the `__init__()` method and try to understand what it is doing
  1. Look at the `deserialize()` method and complete all todos.
      1. Complete the `__to_state_()` and `__to_indices__()` methods.
2. Complete the `reset()` method.
3. Go to the `step()` method and understand its intended behavior.
  1. Complete `__get_next_state_on_action__()`
  2. Complete `__get_reward_for_transition__()`
  3. Complete the `step()` method.


4. Read through the `render()` method to understand how we display the environment in the different situations. 

5. Execute the cell below and make sure there are no errors.

In [ ]:
import numpy as np

class Foolsball(object):

  def __to_state__(self,row,col):
    """Convert from integer state to indices (row,col)."""
    return #Todo

  def __to_indices__(self, state):
    """Convert indices(row,col) to state (single integer)."""
    row = #Todo
    col = #Todo
    return row,col

  def __deserialize__(self,map:list,agent:str,opponent:str, goal:str):
    """Convrt a string representation of a map into a 2D numpy array
    Param map: list of lists of strings representing the player, opponents and goal.
    Param agent: string representing the agent on the map 
    Param opponent: string representing every instance of an opponent player
    Param goal: string representing the location of the goal on the map
    """
    ## Capture dimensions and map.
    self.n_rows = #Todo
    self.n_cols = #Todo
    self.n_states = #Todo
    self.map = #Todo

    ## Store string representations for printing the map, etc.
    self.agent_repr = #Todo
    self.opponent_repr  = #Todo
    self.goal_repr = #Todo

    ## Find initial state, the desired goal state and the state of the opponents. 
    self.init_state = None
    self.goal_state = None
    self.opponents_states = []

    for row in range(self.n_rows):
      for col in range(self.n_cols):

        if map[row][col] == agent:
          # Store the initial state outside the map.
          # This helps in quickly resetting the game to the initial state and
          # also simplifies printing the map independent of the agent's state. 
          self.init_state = #Todo
          self.map[row,col] = ' ' 
        
        elif map[row][col] == opponent:
          #Todo

        elif map[row][col] == goal:
          #Todo

    assert self.init_state is not None, print(f"Map {map} does not specify an agent {agent} location")
    assert self.goal_state is not None,  print(f"Map {map} does not specify a goal {goal} location")
    assert self.opponents_states,  print(f"Map {map} does not specify any opponents {opponent} location")

    return self.init_state


  def __get_next_state_on_action__(self,state,action):
    """Return next state based on current state and action."""
    row, col = self.__to_indices__(state)
    action_to_index_delta = {'n':[-1,0], 'e':[0,+1], 'w':[0,-1], 's':[+1,0]}

    row_delta, col_delta = action_to_index_delta[action]
    new_row , new_col = row+row_delta, col+col_delta

    ## Return current state if next state is invalid
    if #Todo
      return state  

    ## Construct state from new row and col and return it.    
    return #Todo


  def __get_reward_for_transition__(self,state,next_state):
    """ Return the reward based on the transition from current state to next state. """
    ## Transition rejected due to illegal action (move)
    if next_state == state:
      reward = #Todo
    
    ## Goal!
    elif next_state == self.goal_state:
      reward = #Todo
    
    ## Ran into opponent. 
    elif next_state in self.opponents_states:
      reward = self.rewards['opponent']

    ## Made a safe and valid move.   
    else:
      reward = #Todo

    return reward


  def __is_terminal_state__(self, state):
    return (state == self.goal_state) or (state in self.opponents_states) 

  
  def __init__(self,map,agent,opponent,goal):
    """Spawn the world, create variables to track state and actions."""
    # We just need to track the location of the agent (the ball)
    # Everything else is static and so a potential algorithm doesn't 
    # have to look at it. The variable `done` flags terminal states.
    self.state = self.__deserialize__(map,agent,opponent,goal)
    self.done = False
    self.actions = ['n','e','w','s']

    # Set up the rewards
    self.default_rewards = {'unmarked':-1, 'opponent':-5, 'outside':-1, 'goal':+5}
    self.set_rewards(self.default_rewards)



  def reset(self):
    """Reset the environment to its initial state."""
    # There's really just two things we need to reset: the state, which should
    # be reset to the initial state, and the `done` flag which should be 
    # cleared to signal that we are not in a terminal state anymore, even if we 
    # were earlier. 
    self.state = #Todo
    self.done  = #Todo
    return self.state

  
  def set_rewards(self,rewards):
    if not self.state == self.init_state:
      print('Warning: Setting reward while not in initial state! You may want to call reset() first.')
    for key in self.default_rewards:
      assert key in rewards, print(f'Key {key} missing from reward.') 
    self.rewards = rewards

  
  def step(self,action):
    """Simulate state transition based on current state and action received."""
    assert not self.done, \
    print(f'You cannot call step() in a terminal state({self.state}). Check the "done" flag before calling step() to avoid this.')
    next_state = #Todo: Get next state for this (state,action) pair.

    reward = #Todo: Get the reward for the state -> next_state transition.

    done = #Todo: Set the flag if we are in a terminal state.

    self.state, self.done = next_state, done
    
    return next_state, reward, done


  def render(self):
    """Pretty-print the environment and agent."""
    ## Create a copy of the map and change data type to accomodate
    ## 3-character strings
    _map = np.array(self.map, dtype='<U3')

    ## Mark unoccupied positions with special symbol.
    ## And add extra spacing to align all columns.
    for row in range(_map.shape[0]):
      for col in range(_map.shape[1]):
        if _map[row,col] == ' ':
          _map[row,col] = ' + '
        
        elif _map[row,col] == self.opponent_repr: 
          _map[row,col] =  self.opponent_repr + ' '
        
        elif _map[row,col] == self.goal_repr:
          _map[row,col] = ' ' + self.goal_repr + ' '
      
    ## If current state overlaps with the goal state or one of the opponents'
    ## states, susbstitute a distinct marker.
    if self.state == self.goal_state:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' 🏁 '
    elif self.state in self.opponents_states:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' ❗ '
    else:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' ' + self.agent_repr
    
    for row in range(_map.shape[0]):
      for col in range(_map.shape[1]):
        print(f' {_map[row,col]} ',end="")
      print('\n') 
    
    print()



# Step 2: Verify the environment
Execute the two cell below and ensure that there are no runtime error and the rendering happens correctly. You should see output like this:

```
  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  

```

In [ ]:
foolsball = Foolsball(arena, agent, opponent, goal)

In [ ]:
foolsball.render()

# Step 3: Explore the environment
- Run the next cell to play with the environment and score a few goals. 
- If there are any errors you may want to go back and update the code for the `Foolsball` class. 
- Make sure to run the cell containing `foolsball = Foolsball(arena, agent, opponent, goal)` if you update the class.

In [ ]:
## Move: n,s,e,w
## Reset: r
## Exit: x
while True:
  try:
    act = input('>>')

    if act in foolsball.actions:
      print(foolsball.step(act))
      print()
      foolsball.render()
    elif act == 'r':
      print(foolsball.reset())
      print()
      foolsball.render()
    elif act == 'x':
      break
    else:
      print(f'Invalid input:{act}')
  except Exception as e:
    print(e)

# Step 4: Understand the concept of returns
- Complete the `get_return()` function.
- Calculate returns for a few sample paths by running the next few cells

In [ ]:
## Reward and return
path1 = ['e','s','e','s','s','s','e']
path2 = ['s','e','e','s','s','s','e']
path3 = ['s','s','s','e','e','s','e']
path4 = ['s','s','s','s','n','e','e','s','e']

In [ ]:
def get_return(path):
  foolsball.reset()
  foolsball.render()
  
  _return_ = 0
  for act in path: 
    next_state, reward, done = foolsball.step(act)
    foolsball.render()
    _return_ += reward
    
    if done:
      break
    
  print(f'Return (accumulated reward): {_return_}')

In [ ]:
get_return(path1)

In [ ]:
get_return(path2)

In [ ]:
get_return(path3)

In [ ]:
get_return(path4)

# Step 5: Experiment with a different reward structure.
- Does it encourage the agent to take the shortest route?

In [ ]:
## Different reward structure
foolsball.set_rewards({'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5})

In [ ]:
get_return(path1)

In [ ]:
get_return(path4)

# Step 6: Learn about discounted returns
- Get introduced to discounted return as a means to set acceptable time horizons.
$$Discounted\ Return = R_{t_1} + \gamma*R_{t_2} + \gamma^2*R_{t_3} + ... + \gamma^{n-1}*R_{t_n}$$
where $R_{t_k}$  is the reward after step `k` and $\gamma$ is called the discount factor. 
- Complete the code below to implement discounted returns.
- The discount factor $\gamma$ is a hyperparameter (why?) often set to 0.9 
😜
- Run the next few cells to see if discounting indeed has the effect we want (shorter paths have higher rewards)

In [ ]:
def get_discounted_return(path, gamma=0):
  foolsball.reset()
  foolsball.render()
  _return_ = 0
  discount_coeff = 1
  for act in path: 
    #Todo: Run one step and get the state, action, reward tuple
    #Todo: Update discounted return 
    #Todo: Update the discount multiplier to pow(gamma,n-1)
    
    foolsball.render()
    if done:
      break
    
  print(f'Return (accumulated reward): {_return_}')

In [ ]:
HYPER_PARAMS = {'gamma':0.9}

In [ ]:
get_discounted_return(path1, HYPER_PARAMS['gamma'])

In [ ]:
get_discounted_return(path4, HYPER_PARAMS['gamma'])

# Understanding the first bits of terminology.
## State 
- In RL, state refers to information about the environemnt and then agent.
- An RL algorithm inspects the state to decide which action to take.
- Exactly what information gets captured in `state` depends on a few factors:
  - The complexity of the environment: 
    - The number of actors, 
    - the nature of the environment, for example text or images. 
  - The complexity of the algorithm
    - A simple algorithm may only need information about the agent and its immediate surroundings.
    - A more complex algorithm may need information about the whole environment.


## Setup
- In our case we want the algorithm to only know about the location of the agent on the field. 
- We could have included information about the opponents too which would perhaps aid in the decision making but we chose not to.  

- The state therefore is a tuple: (row, col), representing the location of the agent. 
- There are 20 possible values that `state` can take on:
  - `row` can range from 0 through 4
  - `col` can range from 0 through 3

## Implementation details
- The state is actually stored as a single integer that can take on values between 0 and 19.

## Actions
The agents can perfrom actions in an environment.

## Setup
- Our agent can perform one kind of action: navigate up, down, right or left.
- It has 4 actions: 'n', 'e', 'w', 's'.

# Learning from experience
Any RL set up can be modeled as shown below:

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTMDmrmnl_dAyjCOErHPak2gLXmQTgQnVT8gQ&usqp=CAU)

- The agent performs an action in the environment
- The state of the environment and agent change as a result
- The agent receives a reward and the updated state from the environment

## Rewards
- Reward is the signal that an agent receives after it performs an action.
- The reward structure has to be decided by us. 
- The biggest challenge of RL is that reward is often sparse. 

## Set up
- In our case the reward depends on the rules of the game and our goal.
  - If the agent runs into an opponent, the game gets over and the reward is negative (penalizes the agent).
  - If the agent makes it to the goalpost, the game gets over and the reward is positive.
  - if the agent takes the ball out of the field the reward is negative.
  - If the agent makes a valid move what shoud the reward be?

## Implementation
- The default reward structure in our case is  `{'unmarked':-1, 'opponent':-5, 'outside':-1, 'goal':+5}`
- This can be changed at any time by calling `set_rewards()`.
- Taking the ball to an unmarked position seems like a small step towards reaching the goalpost. Why would we then ever want to have a negative reward for this type of manouver?

# Step 7: Formalizing the problem:
- We want the agent to reach the goalpost AND attain the highest **discounted return**.
- This means making safe and efficient moves
  - Running into opponents means game over
  - Repeated 'outsides' means inefficiency
  - Long detours are also inefficient

## The Conundrum
- We already know how to compute the discounted return for a single path.
- We can generate all possible paths and calculate their returns and pick a path with the highest return.

- Alas there are too many paths (4 possible decisions at each step => combinatorial explosion).


## The "Trick"
- Even though there are too many paths, all of them are made up of a smaller number of (state,action) pairs.
- We can calculate the return for each of the 80(=20x4) state action pairs.
- To emphasize, we want to calculate the return for each (state,action) pair, not just the reward.
  - Calculating return means peeking into the future, beyond that (state,action) pair.


## Todo:
- As a precursor to calculating returns for every (state,action) pair, let's try to calculate the reward for every (state,action) pair.

- Understand how the code in the next two cells creates a Pandas table to store the rewards for every (state, action) pair.

- We will cheat a little by using private methods of the `Foolsball` class
  - Use the `__get_next_state_on_action__()` and `__get_reward_for_transition__()` methods to complete the code in the third cell below.
  - Run the fourth cell to view the rewards table. 
  - Notice that rewards for terminal states are kept undefined since no actions are allowed in those states.

